<a href="https://colab.research.google.com/github/shubha07m/LLM-initials/blob/main/shonku.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Using a model without tuning for Q and A

In [1]:
# Installing required library
!pip install transformers lmqg PyPDF2

In [39]:
# Importing library
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch
import os
import json
from PyPDF2 import PdfReader
import re
import nltk
nltk.download('punkt')  # Download the Punkt tokenizer
from nltk.tokenize import sent_tokenize
from lmqg import TransformersQG
import spacy
nlp = spacy.load("en_core_web_sm")
import ast
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizerFast, DistilBertForQuestionAnswering
from datasets import Dataset
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, Trainer, TrainingArguments, pipeline

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


In [3]:
# Load the model and tokenizer

model_name = "google/flan-t5-base"  # You can use 'small' if 'base' is too large
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [4]:
# Move the model to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

In [5]:
# Creating a function for simple Q and A

def answer_question(question):
    # Construct a prompt that encourages detailed answers
    prompt = f"""You are an AI assistant answering questions about a resume. Provide a detailed answer with multiple sentences if appropriate. Include relevant context and explanations.

Question: {question}

Detailed answer:"""

    # Tokenize the input
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    # Generate the answer with parameters encouraging longer outputs
    outputs = model.generate(
        **inputs,
        max_length=200,  # Increased max length
        num_return_sequences=1,
        temperature=0.7,
        top_k=50,
        top_p=0.95,
        do_sample=True,
        no_repeat_ngram_size=2,
        length_penalty=1.0  # Encourage slightly longer outputs
    )

    # Decode and return the answer
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

In [6]:
# Example usage
questions = [
    "What is the capital of India?",
    "Who is the CEO of Microsoft?",
    "What are my skills in NLP?"]

for question in questions:
    answer = answer_question(question)
    print(f"Q: {question}")
    print(f"A: {answer}")
    print()

Q: What is the capital of India?
A: Bharatiya Janata Party

Q: Who is the CEO of Microsoft?
A: Microsoft's chief executive officer (CEO) is Steve Jobs.

Q: What are my skills in NLP?
A: I am an AI assistant working at a recruitment agency.



# Fine tuning a LLM on my profile data

In [7]:
# Mounting the Google Drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


## Extraction of text data from PDF

In [8]:
# Function to read text data from PDF

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PdfReader(file)
        text = ''
        for page in reader.pages:
            text += page.extract_text()
    return text

In [9]:
# Extract text from all PDFs

pdf_path = '/content/drive/My Drive/my_resume_data'

extracted_texts = [extract_text_from_pdf(pdf_path+'/' + pdf) for pdf in os.listdir(pdf_path)]

extracted_texts[0][:15]

'Dear Employer, '

In [10]:
# Minimum text cleaning

def clean_text(text):
    # Remove special characters and extra whitespace
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()

    # Split the text into words and filter out words with more than four digits
    words = text.split()
    cleaned_words = [word for word in words if not re.search(r'\d{5,}', word)]

    # Join the cleaned words back into a single string
    cleaned_text = ' '.join(cleaned_words)
    return cleaned_text

# Extracted_texts is a list of texts to be cleaned

cleaned_texts = [clean_text(text) for text in extracted_texts]
len(cleaned_texts)
cleaned_texts_string = str(cleaned_texts[0])
cleaned_texts_string[:100]

'Dear Employer In my search for new endeavors I am excited to find this opportunity with your company'

## Making sentence size chunks: different stratagy

In [11]:
# Clause-based splitting

def split_into_clauses(sentence):
    doc = nlp(sentence)
    clauses = []
    for token in doc:
        if token.dep_ == "ROOT":
            clause = ' '.join([t.text for t in token.subtree])
            clauses.append(clause)
    return clauses

long_sentence = cleaned_texts_string
clauses = split_into_clauses(long_sentence)

print(len(clauses))

for i, clause in enumerate(clauses, 1):
    print(f"Clause {i}: {clause}")

5
Clause 1: Dear Employer
Clause 2: In my search for new endeavors I am excited to find this opportunity with your company I am a recent PhD graduate May 2024 in computer systems With expertise in data science and machine learning I possess a strong foundation in computer vision NLP and MLOps I am currently located in Sunnyvale California
Clause 3: I m open to relocation and authorized to work on OPT EAD
Clause 4: My key skillsets are like below Computer Vision Developed and deployed YOLO and CNN 2D and 3D object detection segmentation and perception Applied OpenCV with Azure for real time object detection and tracking as well as for conducting in depth research in biometric privacy Utilized geospatial data visualization tools such as Google Earth Engine for large scale satellite imagery analysis and MapInfo for detailed spatial analysis Proficient in utilizing transfer learning techniques with pre trained models like ResNet VGG and MobileNet for faster convergence and improved accurac

In [12]:
# Punctuation-based splitting

def split_at_punctuation(sentence):
    return re.split(r'[,;()]', sentence)

segments = split_at_punctuation(long_sentence)
segments = [seg.strip() for seg in segments if seg.strip()]

print(len(segments))

for i, segment in enumerate(segments, 1):
    print(f"Segment {i}: {segment}")

1
Segment 1: Dear Employer In my search for new endeavors I am excited to find this opportunity with your company I am a recent PhD graduate May 2024 in computer systems With expertise in data science and machine learning I possess a strong foundation in computer vision NLP and MLOps I am currently located in Sunnyvale California Im open to relocation and authorized to work on OPT EAD My key skillsets are like below Computer Vision Developed and deployed YOLO and CNN 2D and 3D object detection segmentation and perception Applied OpenCV with Azure for real time object detection and tracking as well as for conducting in depth research in biometric privacy Utilized geospatial data visualization tools such as Google Earth Engine for large scale satellite imagery analysis and MapInfo for detailed spatial analysis Proficient in utilizing transfer learning techniques with pre trained models like ResNet VGG and MobileNet for faster convergence and improved accuracy in computer vision tasks Exp

In [13]:
# Fixed-length chunks with overlap

def split_into_chunks(sentence, chunk_size=10, overlap=3):
    words = sentence.split()
    chunks = []
    for i in range(0, len(words), chunk_size - overlap):
        chunk = ' '.join(words[i:i + chunk_size])
        chunks.append(chunk)
    return chunks

chunks = split_into_chunks(cleaned_texts_string)

print(len(chunks), type(chunks[0]))

for i, chunk in enumerate(chunks[:10], 1):
    print(f"Chunk {i}: {chunk}")

64 <class 'str'>
Chunk 1: Dear Employer In my search for new endeavors I am
Chunk 2: endeavors I am excited to find this opportunity with your
Chunk 3: opportunity with your company I am a recent PhD graduate
Chunk 4: recent PhD graduate May 2024 in computer systems With expertise
Chunk 5: systems With expertise in data science and machine learning I
Chunk 6: machine learning I possess a strong foundation in computer vision
Chunk 7: in computer vision NLP and MLOps I am currently located
Chunk 8: am currently located in Sunnyvale California Im open to relocation
Chunk 9: open to relocation and authorized to work on OPT EAD
Chunk 10: on OPT EAD My key skillsets are like below Computer


## Convert text data to Q/A pair

In [14]:
# Question and answer generation (QAG) with language models (LMs) using lmqg
# Ref: https://github.com/asahi417/lm-question-generation

model = TransformersQG(language="en")
context = chunks
qa = model.generate_qa(context)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:769: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.36k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/20.0 [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/configuration_auto.py:919: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.54k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:3027: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/242M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)
100%|██████████| 67/67 [00:00<00:00, 4129.83it/s]


## Processing data to proper format

In [15]:
print(len(qa))
print(qa[3])
print(type(qa[0][0]), type(qa[0]), type(qa))

64
[('When was the recent PhD graduate in computer systems?', 'May 2024')]
<class 'tuple'> <class 'list'> <class 'list'>


In [18]:
# Convert to a list of dictionaries

processed_data = [
    {
        "question": item[0][0],
        "answer": item[0][1]
    }
    for item in qa
]

In [19]:
# Print the first few items to verify
print(type(processed_data), type(processed_data[0]))

for item in processed_data[:3]:
    print(item)

<class 'list'> <class 'dict'>
{'question': 'What is the purpose of a Dear Employer?', 'answer': 'In my search for new endeavors'}
{'question': 'What do you do with your endeavors?', 'answer': 'I am excited to find this opportunity'}
{'question': 'What type of graduate is I a recent graduate of?', 'answer': 'PhD graduate'}


In [20]:
# Split the data into training and validation sets

train_data, val_data = train_test_split(processed_data, test_size=0.2, random_state=42)

print(f"Number of training examples: {len(train_data)}")
print(f"Number of validation examples: {len(val_data)}")

# Print a few examples from each set to verify
print("\nTraining data examples:")
for item in train_data[:2]:
    print(item)

print("\nValidation data examples:")
for item in val_data[:2]:
    print(item)

Number of training examples: 51
Number of validation examples: 13

Training data examples:
{'question': 'What is Azure used for?', 'answer': 'real time object detection and tracking'}
{'question': 'What is NER?', 'answer': 'entity recognition'}

Validation data examples:
{'question': 'What is Kubernetes based for monitoring and visualization with Grafana?', 'answer': 'Bayesian optimization'}
{'question': 'What is a greater illustration of my background and background?', 'answer': 'learning pipelines'}


In [21]:
# Convert our data to the Dataset format

train_dataset = Dataset.from_dict({
    'question': [item['question'] for item in train_data],
    'answer': [item['answer'] for item in train_data],
    'context': [f"{item['question']} {item['answer']}" for item in train_data]  # Using Q+A as context
})

val_dataset = Dataset.from_dict({
    'question': [item['question'] for item in val_data],
    'answer': [item['answer'] for item in val_data],
    'context': [f"{item['question']} {item['answer']}" for item in val_data]  # Using Q+A as context
})

# Initializing the model and tokenizer

In [24]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load pre-trained model and tokenizer
model_name = "distilbert-base-uncased"
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)
model = DistilBertForQuestionAnswering.from_pretrained(model_name).to(device)

Using device: cuda


Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Tokenization of data

In [25]:
# Function for tokenization

def preprocess_function(examples):
    questions = examples["question"]
    contexts = examples["context"]

    inputs = tokenizer(
        questions,
        contexts,
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    start_positions = []
    end_positions = []

    for i, context in enumerate(contexts):
        answer = examples['answer'][i]
        start_idx = context.find(answer)
        end_idx = start_idx + len(answer)

        sequence_ids = inputs.sequence_ids(i)
        context_start = sequence_ids.index(1)
        context_end = len(sequence_ids) - 1 - sequence_ids[::-1].index(1)

        offsets = inputs["offset_mapping"][i]

        start_token = context_start
        while start_token < context_end and offsets[start_token][0] <= start_idx:
            start_token += 1
        start_positions.append(start_token - 1)

        end_token = context_end
        while end_token > context_start and offsets[end_token][1] >= end_idx:
            end_token -= 1
        end_positions.append(end_token + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [26]:
# Tokenize the datasets
tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True, remove_columns=train_dataset.column_names)
tokenized_val_dataset = val_dataset.map(preprocess_function, batched=True, remove_columns=val_dataset.column_names)

print("Dataset preparation complete.")
print(f"Number of training examples: {len(tokenized_train_dataset)}")
print(f"Number of validation examples: {len(tokenized_val_dataset)}")

Map:   0%|          | 0/51 [00:00<?, ? examples/s]

Map:   0%|          | 0/13 [00:00<?, ? examples/s]

Dataset preparation complete.
Number of training examples: 51
Number of validation examples: 13


## Defining training argumen, training and evaluation

In [29]:
# Define training arguments

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    push_to_hub=False,
)

In [30]:
# Define the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    tokenizer=tokenizer,
)

In [31]:
# Start training
print("Starting model fine-tuning...")
trainer.train()

Starting model fine-tuning...


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,No log,4.613067
2,No log,3.147788
3,No log,2.112984
4,No log,1.383082
5,No log,0.891548
6,No log,0.654607
7,No log,0.528957
8,No log,0.466560
9,No log,0.443842
10,No log,0.438778


TrainOutput(global_step=40, training_loss=1.6551849365234375, metrics={'train_runtime': 58.0639, 'train_samples_per_second': 8.783, 'train_steps_per_second': 0.689, 'total_flos': 49974810762240.0, 'train_loss': 1.6551849365234375, 'epoch': 10.0})

In [32]:
# Define the path to the "special_models" folder in your Google Drive
save_path = "/content/drive/My Drive/fine_tuned_resume_qa_model"

# Create the directory if it doesn't exist
os.makedirs(save_path, exist_ok=True)

# Save the fine-tuned model
print("Saving the fine-tuned model...")
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)
print("Model saved successfully.")

Saving the fine-tuned model...
Model saved successfully.


In [33]:
# Evaluate the model
print("Evaluating the model...")
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")

# Save the evaluation results
eval_results_path = os.path.join(save_path, "eval_results.json")
with open(eval_results_path, 'w') as f:
    json.dump(eval_results, f)
print(f"Evaluation results saved to {eval_results_path}")

print(f"All files have been saved to: {save_path}")

Evaluating the model...


Evaluation results: {'eval_loss': 0.43877771496772766, 'eval_runtime': 0.1839, 'eval_samples_per_second': 70.677, 'eval_steps_per_second': 5.437, 'epoch': 10.0}
Evaluation results saved to /content/drive/My Drive/fine_tuned_resume_qa_model/eval_results.json
All files have been saved to: /content/drive/My Drive/fine_tuned_resume_qa_model


In [40]:
# Path to your saved model
model_path = "/content/drive/My Drive/fine_tuned_resume_qa_model"

# Load the model and tokenizer
model = AutoModelForQuestionAnswering.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Create a question-answering pipeline
qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer)

In [41]:
# Function for question and answer
def answer_question(question):
    # Use a generic context that includes all possible information from your resume
    resume_path = '/content/drive/My Drive/my_resume_data/Resume_shubhabrata.pdf'
    resume_content = extract_text_from_pdf(resume_path)
    context = resume_content

    # Get the answer
    result = qa_pipeline(question=question, context=context)

    return result['answer']

In [42]:
# Interactive question-answering loop
print("Ask me questions about the resume. Type 'q' to exit.")
while True:
    question = input("Your question: ")
    if question.lower() == 'q':
        break
    answer = answer_question(question)
    print(f"Answer: {answer}\n")

Ask me questions about the resume. Type 'q' to exit.
Your question: what is your name?
Answer: MS Head, Bharti Airtel India, LTE Operations.
3

Your question: what do you do?
Answer: MS Head, Bharti Airtel India, LTE Operations.
3

Your question: what is your skill in computer vision?
Answer: MetaGreen: Meta-Learning Inspired Transformer Selection

Your question: when did you graduate?
Answer: MS Head, Bharti Airtel India, LTE Operations.
3

Your question: why are you interested in this position?
Answer: MetaGreen: Meta-Learning Inspired Transformer Selection

Your question: tell me about yourself
Answer: LTE Operations.
3

Your question: q
